### 1. Importações

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_channels = pd.read_csv(r'C:\Users\Usuario\Documents\TREINAMENTO\PJ_treinamento3_24.0\Previsao-main\Previsao-main\channels.csv')
df_deliveries = pd.read_csv(r'C:\Users\Usuario\Documents\TREINAMENTO\PJ_treinamento3_24.0\Previsao-main\Previsao-main\deliveries.csv')
df_drivers = pd.read_csv(r'C:\Users\Usuario\Documents\TREINAMENTO\PJ_treinamento3_24.0\Previsao-main\Previsao-main\drivers.csv')
df_hubs = pd.read_csv(r'C:\Users\Usuario\Documents\TREINAMENTO\PJ_treinamento3_24.0\Previsao-main\Previsao-main\hubs.csv')
df_orders = pd.read_csv(r'C:\Users\Usuario\Documents\TREINAMENTO\PJ_treinamento3_24.0\Previsao-main\Previsao-main\orders.csv')
df_payments = pd.read_csv(r'C:\Users\Usuario\Documents\TREINAMENTO\PJ_treinamento3_24.0\Previsao-main\Previsao-main\payments.csv')
df_stores = pd.read_csv(r'C:\Users\Usuario\Documents\TREINAMENTO\PJ_treinamento3_24.0\Previsao-main\Previsao-main\stores.csv')

### 2. Unindo as bases

In [3]:
dfmerge1 = pd.merge(df_deliveries, df_drivers, on='driver_id')
dfmerge2 = pd.merge(df_stores, df_hubs, on='hub_id')
df = pd.merge(pd.merge(pd.merge(pd.merge(df_orders, df_channels, on='channel_id'), dfmerge1, on='delivery_order_id'), dfmerge2, on='store_id'), df_payments, on='payment_order_id')

### 3. Limpeza dos dados

#### 3.1. Dropando colunas que não influenciam na análise

In [11]:
df = df.drop(['store_id', 'channel_id', 'payment_order_id', 'delivery_order_id', 'delivery_id', 'driver_id','hub_id','payment_id'], axis=1)

df.columns

Index(['order_id', 'order_status', 'order_amount', 'order_delivery_fee',
       'order_delivery_cost', 'order_created_hour', 'order_created_minute',
       'order_created_day', 'order_created_month', 'order_moment_created',
       'order_moment_accepted', 'order_moment_ready', 'order_moment_collected',
       'order_moment_in_expedition', 'order_moment_delivering',
       'order_moment_delivered', 'order_moment_finished',
       'order_metric_collected_time', 'order_metric_paused_time',
       'order_metric_production_time', 'order_metric_walking_time',
       'order_metric_expediton_speed_time', 'order_metric_transit_time',
       'order_metric_cycle_time', 'channel_type', 'delivery_distance_meters',
       'delivery_status', 'driver_modal', 'driver_type', 'store_segment',
       'store_plan_price', 'store_latitude', 'store_longitude', 'hub_city',
       'hub_latitude', 'hub_longitude', 'payment_amount', 'payment_fee',
       'payment_method', 'payment_status'],
      dtype='object')

In [14]:
df.shape

(287459, 40)

In [15]:
df['payment_status'].value_counts()

payment_status
PAID          287111
CHARGEBACK       335
AWAITING          13
Name: count, dtype: int64

In [16]:
df['order_status'].value_counts()

order_status
FINISHED    287457
CANCELED         2
Name: count, dtype: int64

In [17]:
df['delivery_status'].value_counts()

delivery_status
DELIVERED     287343
DELIVERING       116
Name: count, dtype: int64

In [18]:
df = df[~df['payment_status'].isin(['CHARGEBACK', 'AWAITING'])]
df = df[~df['order_status'].isin(['CANCELED'])]
df = df[~df['delivery_status'].isin(['DELIVERING'])]

In [20]:
df = df.drop(['payment_status', 'order_status', 'delivery_status'], axis=1)

#### 3.2. Valores nulos

tratar

In [22]:
df.isnull().sum()

order_id                                  0
order_amount                              0
order_delivery_fee                        0
order_delivery_cost                    2636
order_created_hour                        0
order_created_minute                      0
order_created_day                         0
order_created_month                       0
order_moment_created                      0
order_moment_accepted                   510
order_moment_ready                     7956
order_moment_collected                17744
order_moment_in_expedition            37754
order_moment_delivering                2338
order_moment_delivered               271199
order_moment_finished                     2
order_metric_collected_time           24682
order_metric_paused_time              39187
order_metric_production_time           7958
order_metric_walking_time             43171
order_metric_expediton_speed_time     10032
order_metric_transit_time              2493
order_metric_cycle_time         

### 4. Análise do tempo de preparo

In [ ]:
colunas = ['order_metric_production_time', 'order_metric_collected_time', 'order_amount', 'payment_amount', 'channel_type']

PROCEDIMENTO
- diferença entre order_moment_created e order_moment_accepted
- Usar weekday() para ver quais dias são finais de semana: isso pode impactar o tempo de preparo
- heatmap 
- bivariável
- outliers (desvpad order_metric_production_time relativamente alto)

### 5. Análise do tempo de expedição

In [ ]:
colunas = ['order_metric_walking_time', 'order_metric_expediton_speed_time', 'driver_modal']

### 6. Análise do tempo de trânsito

In [ ]:
colunas = ['order_metric_transit_time', 'order_delivery_fee', 'delivery_distance_meters', 'hub_city', 'driver_modal', 'driver_type']

- Usar um objeto datatime para verificar se o horário da entrega ('order_moment_delivering') é um horário de pico

verificar se existe uma associação significativa entre duas variáveis categóricas

In [ ]:
from scipy.stats import chi2_contingency

# Crie uma tabela de contingência usando pd.crosstab() para as duas variáveis categóricas
tabela_contingencia = pd.crosstab(df['variavel1'], df['variavel2'])

# Aplique o teste de Qui-Quadrado
chi2, p_valor, _, _ = chi2_contingency(tabela_contingencia)

# Exibe os resultados
print("Estatística Qui-Quadrado:", chi2)
print("Valor-p:", p_valor)